In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Gemma patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): G

In [ ]:
model.config.layerdrop = 0.1  # Simulated Progressive Layer Dropping

model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "v_proj"],  # ✅ Best match for Gemma
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    use_rslora = False,
    loftq_config = None,
)



Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 18 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

# Load IMDB dataset
dataset = load_dataset("imdb", split="train[:20000]")  # Subset for quick training

# We'll create a 'text' field with prompt + label to match SFTTrainer expectations
STATIC_PROMPT = "Classify the sentiment of this review:\n\n"

def preprocess(example):
    review = example["text"]
    label = "positive" if example["label"] == 1 else "negative"
    prompt = f"{STATIC_PROMPT}{review}\nSentiment: {label}"
    return {"text": prompt}

dataset = dataset.map(preprocess)
dataset_text_field = "text"

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
STATIC_PROMPT = "Classify the sentiment of this review:\n\n"

def preprocess(example):
    review = example["text"]
    label = "positive" if example["label"] == 1 else "negative"

    # Full text for training
    full_text = f"{STATIC_PROMPT}{review}\nSentiment: {label}"

    # Tokenize full string
    tokenized = tokenizer(full_text, padding="max_length", truncation=True, max_length=512)

    # For CLM: labels must be same length as input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

dataset = dataset.map(preprocess)


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Important for CLM tasks like this
)


training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    # max_steps=100,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    report_to="none",
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



<ipython-input-7-d446283cbdb6>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 2,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 921,600/2,000,000,000 (0.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.016300
2,2.968600
3,2.998100
4,2.991500
5,2.808100
6,2.968600
7,2.912700
8,3.005400
9,3.024900
10,3.053400


TrainOutput(global_step=2500, training_loss=2.6707463997840883, metrics={'train_runtime': 9081.5902, 'train_samples_per_second': 2.202, 'train_steps_per_second': 0.275, 'total_flos': 1.2182360162304e+17, 'train_loss': 2.6707463997840883, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./unsloth_gemma2b_lora_adapter")
tokenizer.save_pretrained("./unsloth_gemma2b_lora_adapter")


('./unsloth_gemma2b_lora_adapter/tokenizer_config.json',
 './unsloth_gemma2b_lora_adapter/special_tokens_map.json',
 './unsloth_gemma2b_lora_adapter/tokenizer.model',
 './unsloth_gemma2b_lora_adapter/added_tokens.json',
 './unsloth_gemma2b_lora_adapter/tokenizer.json')

In [ ]:
!zip -r unsloth_gemma2b_lora_adapter.zip unsloth_gemma2b_lora_adapter

  adding: unsloth_gemma2b_lora_adapter/ (stored 0%)
  adding: unsloth_gemma2b_lora_adapter/tokenizer_config.json (deflated 96%)
  adding: unsloth_gemma2b_lora_adapter/special_tokens_map.json (deflated 76%)
  adding: unsloth_gemma2b_lora_adapter/adapter_config.json (deflated 53%)
  adding: unsloth_gemma2b_lora_adapter/tokenizer.model (deflated 51%)
  adding: unsloth_gemma2b_lora_adapter/README.md (deflated 66%)
  adding: unsloth_gemma2b_lora_adapter/adapter_model.safetensors (deflated 8%)
  adding: unsloth_gemma2b_lora_adapter/tokenizer.json (deflated 84%)


In [ ]:
from google.colab import files
files.download("unsloth_gemma2b_lora_adapter.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>